In [1]:
import pandas as pd
from functools import reduce
import ast
import pdb

In [2]:
df1 = pd.read_csv('source1.csv')
df2 = pd.read_csv('source2.csv')

In [3]:
#df1.head()

In [4]:
#df2.head()

## How many campaigns spent on more than 4 days?
#### Answer: 3289

In [5]:
spent_rows = df2[df2['spend'] != 0]

In [6]:
campaigns_by_id = list(spent_rows.groupby('campaign_id').apply(lambda x: x['date'].unique()))

In [7]:
five_plus_day_campaigns = list(filter(lambda x: len(x) > 4, campaigns_by_id))

In [8]:
len(five_plus_day_campaigns)

3289

## Which sources reported more "junk" than "noise"?
### Answer: []

## How many source B conversions were there for campaigns targeting NY
### Answer: 110775

In [9]:
ny_campaigns = list(df1.loc[df1['audience'].str.contains('NY')]['campaign_id'])

In [10]:
source_b_ny_count = 0
for campaign in ny_campaigns:
    current = df2.loc[df2['campaign_id'] == campaign]
    for row in current['actions']:
        row = ast.literal_eval(row)
        for item in row:
            if 'B' in item:
                source_b_ny_count += item['B']
    

In [11]:
source_b_ny_count

110775

## What was the total cost per view for all ads of type video, truncated to two decimals?
### Answer: []

In [34]:
video_ads = spent_rows.loc[spent_rows['ad_type'] == 'video']

In [35]:
video_total_views = 0

In [36]:
video_ads.head()

,ad_id,ad_type,campaign_id,date,spend,actions
1,461b37a2-a3a4-4a22-b4b9-a79df7257379,video,375138e8-6e89-483f-a607-f62340c5c49d,2018-01-02,946,"[{""B"": 79, ""action"": ""clicks""}, {""H"": 3, ""acti..."
9,203ec8f7-1bcf-4fe1-9288-14a7fd524ce4,video,375138e8-6e89-483f-a607-f62340c5c49d,2018-01-02,255,"[{""E"": 64, ""action"": ""clicks""}, {""J"": 80, ""act..."
11,e0cbee1f-c25a-4422-beba-d01579b1f3e7,video,53c1c769-6793-4ccf-9eb4-d28cf71b4949,2018-01-02,603,"[{""H"": 91, ""action"": ""conversions""}]"
17,089036aa-1a50-4788-acf2-716eb3321b8f,video,53c1c769-6793-4ccf-9eb4-d28cf71b4949,2018-01-02,534,"[{""J"": 32, ""action"": ""views""}]"
21,f3a246a6-f560-4a1c-8589-62d48ec59693,video,53c1c769-6793-4ccf-9eb4-d28cf71b4949,2018-01-02,438,"[{""F"": 30, ""action"": ""noise""}]"


In [37]:
def get_views(row):
    def helper(r):
        total = 0
        r = ast.literal_eval(r)
        for actn in r:
            if actn["action"] == 'views':
                for key in action.keys():
                    if key != 'action':
                        total += actn[key]
        return total
    video_total_views += helper(row)

In [38]:
video_total_cost = video_ads.head().sum(axis=0)['spend']

In [39]:
video_total_cost

2776

In [40]:
video_ads.apply(get_views, axis=1)

UnboundLocalError: ("local variable 'video_total_views' referenced before assignment", 'occurred at index 1')

In [ ]:
video_total_views

In [32]:
cost_per_view = video_total_cost / video_total_views

2776

In [ ]:
cost_per_view

## What combination of state and hair color had the best (lowest) CPM?
### Answer: IL_pink

In [14]:
total_spend_list = []
def sum_spending(row):
    total_spend_list.append(row[1].sum(axis=0)['spend'])

In [15]:
total_impressions_list = []
def sum_impressions(row):
    total_impressions_list.append(row[1].sum(axis=0)['impressions'])

In [16]:
# get total impressions for each respective state/hair combo / audience from df1
# for reach combo, track its campaigns by summing all their values

df3 = pd.DataFrame(df1.groupby('audience'))

In [17]:
campaigns_by_spend = pd.DataFrame(spent_rows.groupby('campaign_id'))

In [18]:
campaigns_by_spend.apply(sum_spending, axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7442    None
7443    None
7444    None
7445    None
7446    None
7447    None
7448    None
7449    None
7450    None
7451    None
7452    None
7453    None
7454    None
7455    None
7456    None
7457    None
7458    None
7459    None
7460    None
7461    None
7462    None
7463    None
7464    None
7465    None
7466    None
7467    None
7468    None
7469    None
7470    None
7471    None
Length: 7472, dtype: object

In [19]:
campaigns_by_spend['total_spend'] = total_spend_list

In [20]:
campaigns_by_spend.head()

,0,1,total_spend
0,000299d3-976d-4c39-9846-01bbd744c240,ad_id a...,18822
1,0005b8bb-61d3-43ed-924a-54991756ba05,ad_id a...,2687
2,00063a5c-d31a-4f0e-977b-5619cd498ef5,ad_id a...,8055
3,000f4b94-d7b0-43af-ae8a-0d93649b689d,ad_id a...,7633
4,00134b2d-8ae6-4e87-961f-f5323be4c7a7,ad_id a...,9326


In [21]:
int(campaigns_by_spend[campaigns_by_spend[0] == '00134b2d-8ae6-4e87-961f-f5323be4c7a7']['total_spend'])

9326

In [22]:
df3.apply(sum_impressions, axis=1)
df3['total_impressions'] = total_impressions_list

In [23]:
def campaign_sums(campaign_list):
    total = 0
    for c in campaign_list:
        #pdb.set_trace()
        values = campaigns_by_spend[campaigns_by_spend[0] == c]['total_spend'].values
        total += 0 if len(values) == 0 else values[0]
    return total

In [24]:
regions_spends = []
def get_region_sum(row):
    regions_spends.append(campaign_sums(list(row[1]['campaign_id'])))

In [25]:
df3.apply(get_region_sum, axis=1)

KeyboardInterrupt: 

In [ ]:
df3['total_spending'] = regions_spends
df3['CPM'] = 0

In [ ]:
def set_CPM(row):
    row['CPM'] = (row['total_spending']  / row['total_impressions'])* 1000
    return row

In [ ]:
df3 = df3.apply(set_CPM, axis=1)

In [ ]:
#df3.head()

In [ ]:
df3[df3['CPM'] == df3.min(axis=0)['CPM']]